# Verlustprävention an Selbstbedienungskassen im Einzelhandel

## Vergleich der Testdaten und der Trainingsdaten

Ziel: Wir prüfen, ob sich die Trainingsdaten systematisch von den Testdaten unterscheiden.

## 1. Warum ist das wichtig?
Ein verzerrter Testdatensatz könnte dazu führen, dass das trainierte Machine-Learning-Modell auf dem Testdatensatz falsche Ergebnisse liefert.

## 2. Welche Tests wurden durchgeführt?

- **t-Test** für Zahlenwerte (Einkaufssumme, Artikelanzahl, Dauer)
- **Chi²-Test** für Kategorien (Filiale, Kasse, Uhrzeit, Zahlungsmethode)

## 3. Welche Voraussetzungen gelten für den Chi²-Test ?
​
- 1. Nominalskalierte Variablen --> erfüllt
- 2. Unabhängigkeit der Messungen --> erfüllt
- 3. Erwartetet Häufigkeiten pro Zelle > 5 --> erfüllt

## 4. Welche Voraussetzungen gelten für den t-Test ?
​
- 1. Intervall- oder Ratioskalierte Variablen --> erfüllt
- 2. Unabhängigkeit der Messungen --> erfüllt
- 3. Varianzhomogenität --> erfüllt (siehe unten)
- 4. Normalverteilung der abhängingen Variablen --> Annahme: erfüllt wegen Zentralem Grenzwertsatz


## 5. Ergebnisse im Überblick

## Test der Datei "transactions_train_3.parquet" auf statistische Signifikanz 

In [2]:
import pandas as pd
from scipy.stats import ttest_ind, chi2_contingency
# Datei einlesen
train = pd.read_parquet("transactions_train_3.parquet")
test = pd.read_parquet("transactions_test_3.parquet")

In [6]:


# Feature Engineering für beide Sets
for df in [train, test]:
    df['transaction_duration'] = (df['transaction_end'] - df['transaction_start']).dt.total_seconds()
    df['hour_of_day'] = df['transaction_start'].dt.hour

# -------------------------------------
# Spalten definieren
# -------------------------------------
numerical_columns = ['total_amount', 'n_lines', 'customer_feedback', 'transaction_duration']
categorical_columns = ['cash_desk', 'payment_medium', 'store_id', 'hour_of_day']

# -------------------------------------
# Ergebnisse sammeln
# -------------------------------------
results = []

# --- Numerische Spalten ---
for col in numerical_columns:
    train_col = train[col].dropna()
    test_col = test[col].dropna()

    if len(train_col) > 0 and len(test_col) > 0:
        stat, p = ttest_ind(train_col, test_col, equal_var=False)  # Welch’s t-Test

        results.append({
            'Spalte': col,
            'Test': 't-Test',
            'p-Wert': p,
            'Mittelwert (train)': train_col.mean(),
            'Mittelwert (test)': test_col.mean(),
            'Std-Abw (train)': train_col.std(),
            'Std-Abw (test)': test_col.std(),
        })

# --- Kategoriale Spalten ---
# Kombiniere mit Gruppierungsvariable
train['dataset'] = 'train'
test['dataset'] = 'test'
combined = pd.concat([train, test], ignore_index=True)

for col in categorical_columns:
    contingency = pd.crosstab(combined['dataset'], combined[col])

    if contingency.shape[1] > 1:
        chi2, p, _, _ = chi2_contingency(contingency)

        results.append({
            'Spalte': col,
            'Test': 'Chi²-Test',
            'p-Wert': p,
            'Kontingenztabelle': contingency
        })

# -------------------------------------
# Ergebnisse in DataFrame
# -------------------------------------
results_df = pd.DataFrame(results)

# Ausgabe trennen
numerical_results = results_df[results_df['Test'] == 't-Test']
categorical_results = results_df[results_df['Test'] == 'Chi²-Test']

# -------------------------------------
# Ausgabe
# -------------------------------------
print("\nNumerische Spalten (t-Tests):")
display(numerical_results[['Spalte', 'p-Wert', 'Mittelwert (train)', 'Mittelwert (test)', 'Std-Abw (train)', 'Std-Abw (test)']].sort_values(by='p-Wert'))

print("\nKategoriale Spalten (Chi²-Tests):")
for _, row in categorical_results.iterrows():
    print(f"\nSpalte: {row['Spalte']} (Chi²-Test)")
    print(f"p-Wert: {row['p-Wert']:.5f}")
    print("Kontingenztabelle:")
    print(row['Kontingenztabelle'])



Numerische Spalten (t-Tests):


,Spalte,p-Wert,Mittelwert (train),Mittelwert (test),Std-Abw (train),Std-Abw (test)
0,total_amount,0.000196,98.423293,98.994708,109.957256,110.051320
1,n_lines,0.086940,10.578223,10.551809,11.106638,11.043483
2,customer_feedback,0.161025,9.319385,9.307991,1.713751,1.730796
3,transaction_duration,0.464392,77.568515,77.642723,72.926379,72.629600



Kategoriale Spalten (Chi²-Tests):

Spalte: cash_desk (Chi²-Test)
p-Wert: 0.91974
Kontingenztabelle:
cash_desk       0       1       2       3
dataset                                  
test       197284  196943  196988  196288
train      371447  370395  370159  369782

Spalte: payment_medium (Chi²-Test)
p-Wert: 0.00000
Kontingenztabelle:
payment_medium    CASH  CREDIT_CARD
dataset                            
test             91082       696421
train           147559      1334224

Spalte: store_id (Chi²-Test)
p-Wert: 0.00000
Kontingenztabelle:
store_id  3fffea06-686f-42bd-8362-818af86b48a9  \
dataset                                          
test                                    122673   
train                                   232882   

store_id  46e6da32-f4b0-40f3-ada7-fc6ca81ed85d  \
dataset                                          
test                                    197926   
train                                   377446   

store_id  581831fc-6a03-4e38-9025-0a889b7fe542  \

## 4. Fazit

- Die meisten Merkmale zeigen **keine signifikanten Unterschiede** zwischen kontrollierten und nicht kontrollierten Transaktionen.
- Bei `store_id` `payment_medium`und `total_amount` gibt es **jeweils einen Hinweis**, dass sich die Testdaten und die Trainingsdaten signifikant unterscheiden

**Die Testdaten sind nur eingeschränkt geeignet für die trainierten Modelle**
